In [ ]:
# Imports
import zipfile
import os, cv2, numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from keras.applications import VGG16, VGG19, ResNet152
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical

In [ ]:
# Downloads dataset
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

# Extracts datase
local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

--2022-10-29 19:21:32--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 74.125.24.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   286MB/s    in 0.2s    

2022-10-29 19:21:32 (286 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
new_size = (220, 128)

# Merges train dirs
x_train, y_train = [], []
for image_name in os.listdir(train_cats_dir):
    image = cv2.imread(train_cats_dir + '/' + image_name)
    image = cv2.resize(image, new_size)
    x_train.append(image)
    y_train.append(0)

for image_name in os.listdir(train_dogs_dir):
    image = cv2.imread(train_dogs_dir + '/' + image_name)
    image = cv2.resize(image, new_size)
    x_train.append(image)
    y_train.append(1)


# Merges validation dirs
x_validation, y_validation = [], []
for image_name in os.listdir(validation_cats_dir):
    image = cv2.imread(validation_cats_dir + '/' + image_name)
    image = cv2.resize(image, new_size)
    x_validation.append(image)
    y_validation.append(0)

for image_name in os.listdir(validation_dogs_dir):
    image = cv2.imread(validation_dogs_dir + '/' + image_name)
    image = cv2.resize(image, new_size)
    x_validation.append(image)
    y_validation.append(1)

(2000, 128, 220, 3)
(2000,)
(1000, 128, 220, 3)
(1000,)


In [ ]:
# Performs augmentation
aug = ImageDataGenerator(height_shift_range=0.2, width_shift_range=0.2, rotation_range=45)
for i in range(0, len(x_train), 2):
  image = np.array(x_train[i])
  image = np.expand_dims(image, axis = 0)
  imageGen = aug.flow(image)
  image = imageGen.next()
  image = np.squeeze(image, axis = 0)
  x_train.append(image)
  y_train.append(y_train[i])


# Splits dataset
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.30, shuffle=True)
x_train, y_train = np.array(x_train), to_categorical(y_train)
x_test, y_test = np.array(x_test), to_categorical(y_test)

(2100, 128, 220, 3)
(2100,)


In [ ]:
# Instantiates VGG16
vgg16 = VGG16(
    include_top=True,
    weights=None,
    input_shape=(128,220,3),
    classes=2,
    classifier_activation="softmax",
)
vgg16.summary()

# Compiles the model to be used
vgg16.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

# Trains the model
vgg16.fit(x_train, y_train, batch_size=64, shuffle=True, epochs=10, validation_data=(x_validation, y_validation))

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 220, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 220, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 220, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 110, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 110, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 110, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 55, 128)       0     

In [ ]:
# Instantiates VGG19
vgg19 = VGG19(
    include_top=True,
    weights=None,
    input_shape=(128,220,3),
    classes=2,
    classifier_activation="softmax",
)
vgg19.summary()

# Compiles the model to be used
vgg19.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

# Trains the model
vgg19.fit(x_train, y_train, batch_size=64, shuffle=True, epochs=10, validation_data=(x_validation, y_validation))

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 220, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 220, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 220, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 110, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 110, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 110, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 55, 128)       0     

In [ ]:
# Instantiates ResNet152
resNet152 = ResNet152(
    include_top=True,
    weights=None,
    input_shape=(128,220,3),
    classes=2,
    classifier_activation="softmax",
)
resNet152.summary()

# Compiles the model to be used
resNet152.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

# Trains the model
resNet152.fit(x_train, y_train, batch_size=64, shuffle=True, epochs=10, validation_data=(x_validation, y_validation))

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 220, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 226, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 110, 64)  9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 110, 64)  256         ['conv1_conv[0][0]']     